In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [3]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [4]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [5]:
NUM_OF_COL = 40

In [6]:
SkipCol = {}
SkipCol[18] = True
SkipCol

{18: True}

In [7]:
y = df_train['label']
y = y.values
y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [8]:
df_test = pd.read_csv('sample_solution.csv')


In [9]:
y.shape

(30336,)

In [10]:
import numpy as np
XTrain = np.load('XTrain.npy', allow_pickle=True)
XTrain.shape

(30336, 383)

In [11]:
XTest = np.load('XTest.npy', allow_pickle=True)
XTest.shape

(10000, 383)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, y, test_size=0.2, random_state=42)

In [13]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

In [14]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'
bst = lgb.train(param, train_set, 5, valid_sets=valid_set, early_stopping_rounds=5)
bst.best_iteration 

[1]	valid_0's auc: 0.827568
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.82969
[3]	valid_0's auc: 0.830194
[4]	valid_0's auc: 0.829432
[5]	valid_0's auc: 0.828792
Did not meet early stopping. Best iteration is:
[3]	valid_0's auc: 0.830194


3

In [15]:
bst.best_score['valid_0']['auc']

0.8301936380621019

In [28]:
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = np.log(1 - best_score)
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [29]:
from hyperopt import hp

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

space

{'boosting_type': <hyperopt.pyll.base.Apply at 0x1c78f6feac8>,
 'num_leaves': <hyperopt.pyll.base.Apply at 0x1c78f6fe9b0>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x1c78f6fec88>,
 'subsample_for_bin': <hyperopt.pyll.base.Apply at 0x1c7869b8470>,
 'min_child_samples': <hyperopt.pyll.base.Apply at 0x1c7869b8ba8>,
 'reg_alpha': <hyperopt.pyll.base.Apply at 0x1c7869b8358>,
 'reg_lambda': <hyperopt.pyll.base.Apply at 0x1c7869b8d30>,
 'colsample_bytree': <hyperopt.pyll.base.Apply at 0x1c78f6ea4a8>,
 'metric': 'auc'}

In [30]:
from hyperopt import Trials
# Trials object to track progress
bayes_trials = Trials()

In [ ]:
from hyperopt import fmin
from hyperopt import tpe

MAX_EVALS = 500

# Optimize
bestDict = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
bestDict

[1]	valid_0's auc: 0.830891                                             
[2]	valid_0's auc: 0.831833                                             
[3]	valid_0's auc: 0.833028                                             
[4]	valid_0's auc: 0.830605                                             
[5]	valid_0's auc: 0.830818                                             
[6]	valid_0's auc: 0.83142                                              
[7]	valid_0's auc: 0.832203                                             
[8]	valid_0's auc: 0.831669                                             
[9]	valid_0's auc: 0.831826                                             
[10]	valid_0's auc: 0.831937                                            
[11]	valid_0's auc: 0.832121                                            
[12]	valid_0's auc: 0.831426                                            
[13]	valid_0's auc: 0.831325                                            
[14]	valid_0's auc: 0.831474                       

[13]	valid_0's auc: 0.827417                                            
[14]	valid_0's auc: 0.827321                                            
[15]	valid_0's auc: 0.827139                                            
[16]	valid_0's auc: 0.827642                                            
[17]	valid_0's auc: 0.826958                                            
[18]	valid_0's auc: 0.826947                                            
[19]	valid_0's auc: 0.826799                                            
[20]	valid_0's auc: 0.826794                                            
[21]	valid_0's auc: 0.826569                                            
[22]	valid_0's auc: 0.826919                                            
[23]	valid_0's auc: 0.827007                                            
[24]	valid_0's auc: 0.826911                                            
[25]	valid_0's auc: 0.82684                                             
[26]	valid_0's auc: 0.826908                       

[12]	valid_0's auc: 0.832513                                            
[13]	valid_0's auc: 0.832479                                            
[14]	valid_0's auc: 0.832087                                            
[15]	valid_0's auc: 0.83177                                             
Early stopping, best iteration is:                                      
[10]	valid_0's auc: 0.833432
[1]	valid_0's auc: 0.830854                                             
Training until validation scores don't improve for 5 rounds             
[2]	valid_0's auc: 0.830986                                             
[3]	valid_0's auc: 0.831214                                             
[4]	valid_0's auc: 0.830456                                             
[5]	valid_0's auc: 0.831239                                             
[6]	valid_0's auc: 0.832536                                             
[7]	valid_0's auc: 0.833127                                             
[8]	valid_0's auc: 0.8

[5]	valid_0's auc: 0.827096                                             
[6]	valid_0's auc: 0.826565                                             
[7]	valid_0's auc: 0.82744                                              
[8]	valid_0's auc: 0.828086                                             
[9]	valid_0's auc: 0.827809                                             
[10]	valid_0's auc: 0.827758                                            
[11]	valid_0's auc: 0.827831                                            
[12]	valid_0's auc: 0.827718                                            
[13]	valid_0's auc: 0.828002                                            
Early stopping, best iteration is:                                      
[8]	valid_0's auc: 0.828086
[1]	valid_0's auc: 0.826881                                             
Training until validation scores don't improve for 5 rounds             
[2]	valid_0's auc: 0.826607                                             
[3]	valid_0's auc: 0.82

[45]	valid_0's auc: 0.831464                                            
[46]	valid_0's auc: 0.831003                                            
[47]	valid_0's auc: 0.830729                                            
[48]	valid_0's auc: 0.830841                                            
[49]	valid_0's auc: 0.830855                                            
[50]	valid_0's auc: 0.830858                                            
[1]	valid_0's auc: 0.825725                                             
Training until validation scores don't improve for 5 rounds             
[2]	valid_0's auc: 0.826535                                             
[3]	valid_0's auc: 0.82635                                              
[4]	valid_0's auc: 0.826037                                             
[5]	valid_0's auc: 0.825432                                             
[6]	valid_0's auc: 0.825634                                             
[7]	valid_0's auc: 0.825766                        

...

[11]	valid_0's auc: 0.831619                                            
[12]	valid_0's auc: 0.831062                                            
[13]	valid_0's auc: 0.831127                                            
[14]	valid_0's auc: 0.831927                                            
[15]	valid_0's auc: 0.832                                               
Early stopping, best iteration is:                                      
[10]	valid_0's auc: 0.832078
[1]	valid_0's auc: 0.83249                                              
Training until validation scores don't improve for 5 rounds             
[2]	valid_0's auc: 0.834602                                             
[3]	valid_0's auc: 0.833613                                             
[4]	valid_0's auc: 0.831034                                             
[5]	valid_0's auc: 0.832697                                             
[6]	valid_0's auc: 0.831944                                             
[7]	valid_0's auc: 0.8

[18]	valid_0's auc: 0.829434                                            
[19]	valid_0's auc: 0.829933                                            
[20]	valid_0's auc: 0.829159                                            
[21]	valid_0's auc: 0.829077                                            
[22]	valid_0's auc: 0.829355                                            
[23]	valid_0's auc: 0.829137                                            
[24]	valid_0's auc: 0.828757                                            
[25]	valid_0's auc: 0.828411                                            
[26]	valid_0's auc: 0.827916                                            
[27]	valid_0's auc: 0.827867                                            
[28]	valid_0's auc: 0.827965                                            
[29]	valid_0's auc: 0.828107                                            
[30]	valid_0's auc: 0.828575                                            
[31]	valid_0's auc: 0.828488                       

[14]	valid_0's auc: 0.832053                                            
[15]	valid_0's auc: 0.831802                                            
[16]	valid_0's auc: 0.832659                                            
[17]	valid_0's auc: 0.833065                                            
[18]	valid_0's auc: 0.832815                                            
[19]	valid_0's auc: 0.833083                                            
[20]	valid_0's auc: 0.832907                                            
[21]	valid_0's auc: 0.833051                                            
[22]	valid_0's auc: 0.832619                                            
[23]	valid_0's auc: 0.83262                                             
[24]	valid_0's auc: 0.832859                                            
Early stopping, best iteration is:                                      
[19]	valid_0's auc: 0.833083
[1]	valid_0's auc: 0.826921                                             
[2]	valid_0's auc: 0.8

[10]	valid_0's auc: 0.831872                                            
Early stopping, best iteration is:                                      
[5]	valid_0's auc: 0.835953
[1]	valid_0's auc: 0.827748                                             
Training until validation scores don't improve for 5 rounds             
[2]	valid_0's auc: 0.828261                                             
[3]	valid_0's auc: 0.828417                                             
[4]	valid_0's auc: 0.829075                                             
[5]	valid_0's auc: 0.829234                                             
[6]	valid_0's auc: 0.828985                                             
[7]	valid_0's auc: 0.828413                                             
[8]	valid_0's auc: 0.828031                                             
[9]	valid_0's auc: 0.827026                                             
[10]	valid_0's auc: 0.82711                                             
Early stopping, best it

[2]	valid_0's auc: 0.832759                                             
[3]	valid_0's auc: 0.833631                                             
[4]	valid_0's auc: 0.833445                                             
[5]	valid_0's auc: 0.832539                                             
[6]	valid_0's auc: 0.833683                                             
[7]	valid_0's auc: 0.833395                                             
[8]	valid_0's auc: 0.832602                                             
[9]	valid_0's auc: 0.832648                                             
[10]	valid_0's auc: 0.831876                                            
[11]	valid_0's auc: 0.831145                                            
Early stopping, best iteration is:                                      
[6]	valid_0's auc: 0.833683
[1]	valid_0's auc: 0.831873                                             
Training until validation scores don't improve for 5 rounds             


In [21]:
from hyperopt import space_eval
best = space_eval(space, bestDict)
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
best['metric'] = 'auc'
bestModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)

[1]	valid_0's auc: 0.834583
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.836436
[3]	valid_0's auc: 0.83721
[4]	valid_0's auc: 0.835234
[5]	valid_0's auc: 0.83371
[6]	valid_0's auc: 0.83332
[7]	valid_0's auc: 0.832925
[8]	valid_0's auc: 0.832461
Early stopping, best iteration is:
[3]	valid_0's auc: 0.83721


In [22]:
bestModel.save_model('model.txt', num_iteration=bestModel.best_iteration)

In [23]:
bestModel.num_trees()

3

In [24]:
best

{'boosting_type': 'goss',
 'colsample_bytree': 0.8104926452200226,
 'learning_rate': 0.03906143402679307,
 'metric': 'auc',
 'min_child_samples': 90,
 'num_leaves': 74,
 'reg_alpha': 0.9113579025036644,
 'reg_lambda': 0.9165121558146241,
 'subsample_for_bin': 120000}

In [25]:
from lightgbm import LGBMClassifier

gb_clf = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
gb_clf.fit(XTrain, y)

KeyError: 'colsample_by_tree'

In [ ]:
probs = gb_clf.predict_proba(XTest, num_iteration=bestModel.best_iteration)
YTest = probs[:, 0]
YTest

In [ ]:
np.max(YTest)

In [ ]:
df_test['Predicted'] = YTest
df_test

In [ ]:
df_test.to_csv('test.csv', index=False)

In [ ]:
from lightgbm import LGBMModel
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

clf = LGBMModel()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10) 
                 }

randm = RandomizedSearchCV(estimator=gb_clf, param_distributions = parameters, 
                           cv = 5, n_iter = 10, n_jobs=-1)
randm.fit(XTrain, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

gb_clf = randm.best_estimator_